In [1]:
import os
import numpy as np
import pandas as pd

/tmp/ipykernel_102/2916776140.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:


q4_data_file = "ncu_report_full_q4_2layers_utilization_full.csv"
q8_data_file = "ncu_report_full_q8_2layers_utilization_full.csv"



In [29]:
def is_interested(col_name):
    return True
    
def get_aggregated_stat(source_data_path):
    print("=== Reading data:", source_data_path)
    source_data = pd.read_csv(source_data_path, thousands=',')
    cols = source_data.columns
    selected_cols = cols[[8]]
    appending_cols = [col for col in cols[18:] if is_interested(col)]
    print("Interested metrics in data file:",len(appending_cols))
    source_stat = source_data.loc[:,list(selected_cols)+list(appending_cols)]
    print(source_stat.info())
    # for col in source_stat.columns[:20]:
    #     print("- ", source_stat[col].dtypes)
    print("Data types:", source_stat.iloc[:,:].dtypes)
    # res = source_stat.iloc[:,:].groupby(['Demangled Name'], sort=True).mean()
    print("=== End of Processing:", source_data_path)
    return source_stat
    # return res


In [16]:
! head -n 2 ./ncu_report_full_q4_2layers_utilization.csv | awk '{for(i=1;i<=18;i++) printf "%s ", $i; print ""}'

﻿ID,Time,API Call ID,Estimated Speedup,"Runtime Improvement""""(0)","Issues Detected""""(850)",Function Name,Mangled Name,Demangled Name,Process,Thread ID,Device Name,CUprogram,CUfunction,Grid Offset,Grid Size,Block Size,Grid Dimensions,gpc__cycles_elapsed.max [cycle],gpu__time_duration.sum [usecond],sm__throughput.avg.pct_of_peak_sustained_elapsed [%],gpu__compute_memory_throughput.avg.pct_of_peak_sustained_elapsed 
0,2024-Feb-13 13:21:25,20399,0.00,0.00,2,mul_mat_vec_q,_Z13mul_mat_vec_qILi32ELi4E10block_q4_0Li2EXadL_ZN43_INTERNAL_642294f9_12_ggml_cuda_cu_fe7a8db317vec_dot_q4_0_q8_1EPKvPK10block_q8_1RKiEEEvS3_S3_Pfii,"void mul_mat_vec_q<(int)32, (int)4, block_q4_0, (int)2, &vec_dot_q4_0_q8_1>(const void *, const void *, float *, int, int)",[206719] main,206719,NVIDIA GeForce 


In [30]:
q4_source_stat = get_aggregated_stat(q4_data_file)


=== Reading data: ncu_report_full_q4_2layers_utilization_full.csv
Interested metrics in data file: 114393
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 114394 entries, Demangled Name to tpc__warps_active_shader_cs_realtime.sum.per_second [warp/nsecond]
dtypes: float64(55895), int64(58455), object(44)
memory usage: 87.3+ MB
None
Data types: Demangled Name                                                           object
gpc__cycles_elapsed.max [cycle]                                           int64
gpu__time_duration.sum [usecond]                                        float64
sm__throughput.avg.pct_of_peak_sustained_elapsed [%]                    float64
gpu__compute_memory_throughput.avg.pct_of_peak_sustained_elapsed [%]    float64
                                                                         ...   
tpc__warps_active_shader_cs_realtime.sum.per_cycle_active [warp]        float64
tpc__warps_active_shader_cs_realtime.sum.per_cycle_elapsed [warp

In [33]:
drop_cols = []
for col in q4_source_stat.columns[1:]:
    if q4_source_stat[col].dtype == "object":
        # print(col)
        drop_cols.append(col)

In [36]:
selected_q4_source_stat = q4_source_stat.drop(drop_cols, axis=1)

In [39]:
q4_agg = selected_q4_source_stat.groupby(['Demangled Name'], sort=True).mean()

In [40]:
q8_source_stat = get_aggregated_stat(q8_data_file)
drop_cols = []
for col in q8_source_stat.columns[1:]:
    if q8_source_stat[col].dtype == "object":
        # print(col)
        drop_cols.append(col)
selected_q8_source_stat = q8_source_stat.drop(drop_cols, axis=1)
q8_agg = selected_q8_source_stat.groupby(['Demangled Name'], sort=True).mean()

=== Reading data: ncu_report_full_q8_2layers_utilization_full.csv
Interested metrics in data file: 114393
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 114394 entries, Demangled Name to tpc__warps_active_shader_cs_realtime.sum.per_second [warp/nsecond]
dtypes: float64(56282), int64(58068), object(44)
memory usage: 87.3+ MB
None
Data types: Demangled Name                                                           object
gpc__cycles_elapsed.max [cycle]                                           int64
gpu__time_duration.sum [usecond]                                        float64
sm__throughput.avg.pct_of_peak_sustained_elapsed [%]                    float64
gpu__compute_memory_throughput.avg.pct_of_peak_sustained_elapsed [%]    float64
                                                                         ...   
tpc__warps_active_shader_cs_realtime.sum.per_cycle_active [warp]        float64
tpc__warps_active_shader_cs_realtime.sum.per_cycle_elapsed [warp

In [42]:
# pd.options.display.float_format = '{:.0f}'.format
pd.options.display.float_format = '{:,.2f}'.format


In [48]:
diff = q8_agg.sub(q4_agg, fill_value=0.0)
diff.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, k_compute_batched_ptrs(const __half *, const __half *, char *, const void **, void **, long, long, long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, long, long) to void soft_max_f32<(bool)1, (int)64, (int)64>(const float *, const float *, float *, int, int, float)
Columns: 114471 entries, c2clink__enabled_mask to tpc__warps_active_shader_cs_realtime.sum.per_second [warp/nsecond]
dtypes: float64(114471)
memory usage: 18.3+ MB


In [52]:
# Find which col & row have any non-zero values:
nonzero_diff = diff.loc[:,(diff!=0).any()].loc[(diff!=0).any(axis=1)]

In [60]:
nonzero_diff

,device__attribute_architecture,device__attribute_async_engine_count,device__attribute_can_map_host_memory,device__attribute_can_tex2d_gather,device__attribute_can_use_64_bit_stream_mem_ops,device__attribute_can_use_host_pointer_for_registered_mem,device__attribute_can_use_stream_wait_value_nor,device__attribute_chip,device__attribute_clock_rate,device__attribute_compute_capability_major,...,tpc__warps_active_shader_cs_realtime.sum.peak_sustained_elapsed.per_second [warp/nsecond],tpc__warps_active_shader_cs_realtime.sum.peak_sustained_frame [warp],tpc__warps_active_shader_cs_realtime.sum.peak_sustained_frame.per_second [warp/nsecond],tpc__warps_active_shader_cs_realtime.sum.peak_sustained_region [warp],tpc__warps_active_shader_cs_realtime.sum.peak_sustained_region.per_second [warp/nsecond],tpc__warps_active_shader_cs_realtime.sum.per_cycle_active [warp],tpc__warps_active_shader_cs_realtime.sum.per_cycle_elapsed [warp],tpc__warps_active_shader_cs_realtime.sum.per_cycle_in_frame [warp],tpc__warps_active_shader_cs_realtime.sum.per_cycle_in_region [warp],tpc__warps_active_shader_cs_realtime.sum.per_second [warp/nsecond]
Demangled Name,,,,,,,,,,,,,,,,,,,,,
"k_compute_batched_ptrs(const __half *, const __half *, char *, const void **, void **, long, long, long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, long, long)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,4.04,"3,653.33",4.04,"3,653.33",4.04,0.02,0.02,0.02,0.02,0.04
"k_sum_rows_f32(const float *, float *, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-0.23,"95,200.00",-0.23,"95,200.00",-0.23,-0.01,-0.01,-0.01,-0.01,0.00
"quantize_q8_1(const float *, void *, int, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.89,"276,986.76",1.89,"276,986.76",1.89,-6.99,-6.99,-6.99,-6.99,-10.99
"silu_f32(const float *, float *, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.40,"-32,592.00",0.40,"-32,592.00",0.40,5.37,5.37,5.37,5.37,8.69
turing_h1688gemm_256x64_ldg8_stages_32x1_tn,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.08,"449,104.00",1.08,"449,104.00",1.08,-0.28,-0.28,-0.28,-0.28,-0.36
"void convert_unary<__half, float>(const void *, T2 *, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,6.21,"-27,536.00",6.21,"-27,536.00",6.21,2.23,2.23,2.23,2.23,3.88
"void convert_unary<float, __half>(const void *, T2 *, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-0.88,"30,384.00",-0.88,"30,384.00",-0.88,4.17,4.17,4.17,4.17,6.76
"void cpy_f32_f16<&cpy_1_f32_f16>(const char *, char *, int, int, int, int, int, int, int, int, int, int, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-2.09,"53,520.00",-2.09,"53,520.00",-2.09,-0.47,-0.47,-0.47,-0.47,-0.79
"void cpy_f32_f16<&cpy_1_f32_f32>(const char *, char *, int, int, int, int, int, int, int, int, int, int, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-5.00,"123,840.00",-5.00,"123,840.00",-5.00,0.51,0.51,0.51,0.51,0.61


In [57]:
with open("metrics.txt", "w") as f:
    for col in nonzero_diff.columns[:-1]:
        f.write(str(col))
        f.write(",")
    f.write(nonzero_diff.columns[-1])

In [118]:
threshold = 0.5
# Relative Percentage Difference (RPD)
# https://en.wikipedia.org/wiki/Relative_change
q4_q8_col = q4_agg.columns.intersection(q8_agg.columns)
intersection = nonzero_diff.columns.intersection(q4_q8_col)
selected_q4_agg_col = q4_agg.columns.intersection(intersection)
selected_q8_agg_col = q8_agg.columns.intersection(intersection)


print(len(selected_q4_agg_col), len(selected_q8_agg_col))

75998 75998


In [88]:
"gpu__time_end.avg [msecond]" in list(q8_agg.columns)

True

In [89]:
"gpu__time_end.avg [msecond]" in list(nonzero_diff.columns)

True

In [82]:
len(nonzero_diff.columns)

76198

In [87]:
set(nonzero_diff.columns) - set(selected_q4_agg_col)

{'gpu__time_end.avg [msecond]',
 'gpu__time_end.max [msecond]',
 'gpu__time_end.min [msecond]',
 'gpu__time_end.sum [msecond]',
 'gpu__time_start.avg [msecond]',
 'gpu__time_start.max [msecond]',
 'gpu__time_start.min [msecond]',
 'gpu__time_start.sum [msecond]',
 'l1tex__m_xbar2l1tex_read_bytes.avg [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes.max [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes.min [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes_mem_lg_op_ld.avg [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes_mem_lg_op_ld.max [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes_mem_lg_op_ld.min [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes_pipe_lsu.avg [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes_pipe_lsu.max [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes_pipe_lsu.min [Mbyte]',
 'l1tex__t_bytes_lookup_miss.avg [Mbyte]',
 'l1tex__t_bytes_lookup_miss.max [Mbyte]',
 'l1tex__t_bytes_lookup_miss.min [Mbyte]',
 'l1tex__t_bytes_pipe_lsu_lookup_miss.avg [Mbyte]',
 'l1tex__t_bytes_pipe_lsu_lookup_miss.max [Mbyte]',
 'l1tex__t_byt

In [59]:
selected_q4_agg

Index(['gpc__cycles_elapsed.max [cycle]', 'gpu__time_duration.sum [usecond]',
       'sm__throughput.avg.pct_of_peak_sustained_elapsed [%]',
       'gpu__compute_memory_throughput.avg.pct_of_peak_sustained_elapsed [%]',
       'device__attribute_architecture',
       'device__attribute_async_engine_count',
       'device__attribute_can_map_host_memory',
       'device__attribute_can_tex2d_gather',
       'device__attribute_can_use_64_bit_stream_mem_ops',
       'device__attribute_can_use_host_pointer_for_registered_mem',
       ...
       'tpc__warps_active_shader_cs_realtime.sum.peak_sustained_elapsed.per_second [warp/nsecond]',
       'tpc__warps_active_shader_cs_realtime.sum.peak_sustained_frame [warp]',
       'tpc__warps_active_shader_cs_realtime.sum.peak_sustained_frame.per_second [warp/nsecond]',
       'tpc__warps_active_shader_cs_realtime.sum.peak_sustained_region [warp]',
       'tpc__warps_active_shader_cs_realtime.sum.peak_sustained_region.per_second [warp/nsecond]',
      

In [94]:
intersection = set(list(nonzero_diff.columns))

In [105]:
q4_idx = [c for c in q4_agg.columns if c in intersection]
q8_idx = [c for c in q8_agg.columns if c in intersection]

In [106]:
len(q4_agg.columns)

114350

In [107]:
len(q8_agg.columns)

114350

In [108]:
len(q4_idx)

76077

In [109]:
len(q8_idx)

76119

In [110]:
set(q8_idx) - set(q4_idx)

{'gpu__time_end.avg [msecond]',
 'gpu__time_end.max [msecond]',
 'gpu__time_end.min [msecond]',
 'gpu__time_end.sum [msecond]',
 'gpu__time_start.avg [msecond]',
 'gpu__time_start.max [msecond]',
 'gpu__time_start.min [msecond]',
 'gpu__time_start.sum [msecond]',
 'l1tex__m_xbar2l1tex_read_bytes.avg [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes.max [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes.min [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes_mem_lg_op_ld.avg [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes_mem_lg_op_ld.max [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes_mem_lg_op_ld.min [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes_pipe_lsu.avg [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes_pipe_lsu.max [Mbyte]',
 'l1tex__m_xbar2l1tex_read_bytes_pipe_lsu.min [Mbyte]',
 'l1tex__t_bytes_lookup_miss.avg [Mbyte]',
 'l1tex__t_bytes_lookup_miss.max [Mbyte]',
 'l1tex__t_bytes_lookup_miss.min [Mbyte]',
 'l1tex__t_bytes_pipe_lsu_lookup_miss.avg [Mbyte]',
 'l1tex__t_bytes_pipe_lsu_lookup_miss.max [Mbyte]',
 'l1tex__t_byt

In [111]:
"gpu__time_end.avg [msecond]" in q4_agg.columns # Reason: gpu__time_end.avg [usecond] vs gpu__time_end.avg [msecond]

False

In [119]:
# selected_q4_agg = q4_agg[q4_idx]
# selected_q8_agg = q8_agg[q8_idx]

selected_q4_agg = q4_agg[selected_q4_agg_col]
selected_q8_agg = q8_agg[selected_q8_agg_col]

In [120]:
selected_q4_agg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, k_compute_batched_ptrs(const __half *, const __half *, char *, const void **, void **, long, long, long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, long, long) to void soft_max_f32<(bool)1, (int)64, (int)64>(const float *, const float *, float *, int, int, float)
Columns: 75998 entries, gpc__cycles_elapsed.max [cycle] to tpc__warps_active_shader_cs_realtime.sum.per_second [warp/nsecond]
dtypes: float64(75998)
memory usage: 12.2+ MB


In [121]:
selected_q8_agg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, k_compute_batched_ptrs(const __half *, const __half *, char *, const void **, void **, long, long, long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, long, long) to void soft_max_f32<(bool)1, (int)64, (int)64>(const float *, const float *, float *, int, int, float)
Columns: 75998 entries, gpc__cycles_elapsed.max [cycle] to tpc__warps_active_shader_cs_realtime.sum.per_second [warp/nsecond]
dtypes: float64(75998)
memory usage: 11.6+ MB


In [124]:
selected_nonzero_diff = nonzero_diff[intersection]

In [125]:
print(len(selected_nonzero_diff.columns), len(selected_q4_agg.columns), len(selected_q8_agg.columns))

75998 75998 75998


In [183]:
# l1tex__data_pipe_lsu_wavefronts_mem_lg_cmd_write.max can < 0 (inflow and outflow) (Level 1 (L1)/Texture Cache, l1tex) LSU (Load Store Unit)

stat = selected_nonzero_diff.div( abs(selected_q4_agg.add(selected_q8_agg, fill_value=0.0)), fill_value=0.0)
stat

,device__attribute_architecture,device__attribute_async_engine_count,device__attribute_can_map_host_memory,device__attribute_can_tex2d_gather,device__attribute_can_use_64_bit_stream_mem_ops,device__attribute_can_use_host_pointer_for_registered_mem,device__attribute_can_use_stream_wait_value_nor,device__attribute_chip,device__attribute_clock_rate,device__attribute_compute_capability_major,...,tpc__warps_active_shader_cs_realtime.sum.peak_sustained_elapsed.per_second [warp/nsecond],tpc__warps_active_shader_cs_realtime.sum.peak_sustained_frame [warp],tpc__warps_active_shader_cs_realtime.sum.peak_sustained_frame.per_second [warp/nsecond],tpc__warps_active_shader_cs_realtime.sum.peak_sustained_region [warp],tpc__warps_active_shader_cs_realtime.sum.peak_sustained_region.per_second [warp/nsecond],tpc__warps_active_shader_cs_realtime.sum.per_cycle_active [warp],tpc__warps_active_shader_cs_realtime.sum.per_cycle_elapsed [warp],tpc__warps_active_shader_cs_realtime.sum.per_cycle_in_frame [warp],tpc__warps_active_shader_cs_realtime.sum.per_cycle_in_region [warp],tpc__warps_active_shader_cs_realtime.sum.per_second [warp/nsecond]
Demangled Name,,,,,,,,,,,,,,,,,,,,,
"k_compute_batched_ptrs(const __half *, const __half *, char *, const void **, void **, long, long, long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, long, long)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.02,0.02,0.03
"k_sum_rows_f32(const float *, float *, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-0.00,0.01,-0.00,0.01,-0.00,-0.01,-0.01,-0.01,-0.01,0.00
"quantize_q8_1(const float *, void *, int, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.03,0.00,0.03,0.00,-0.04,-0.04,-0.04,-0.04,-0.04
"silu_f32(const float *, float *, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,-0.00,0.00,-0.00,0.00,0.01,0.01,0.01,0.01,0.01
turing_h1688gemm_256x64_ldg8_stages_32x1_tn,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.01,0.00,0.01,0.00,-0.00,-0.00,-0.00,-0.00,-0.00
"void convert_unary<__half, float>(const void *, T2 *, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,-0.00,0.00,-0.00,0.00,0.03,0.03,0.03,0.03,0.03
"void convert_unary<float, __half>(const void *, T2 *, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-0.00,0.00,-0.00,0.00,-0.00,0.05,0.05,0.05,0.05,0.05
"void cpy_f32_f16<&cpy_1_f32_f16>(const char *, char *, int, int, int, int, int, int, int, int, int, int, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-0.00,0.01,-0.00,0.01,-0.00,-0.02,-0.02,-0.02,-0.02,-0.02
"void cpy_f32_f16<&cpy_1_f32_f32>(const char *, char *, int, int, int, int, int, int, int, int, int, int, int)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-0.00,0.02,-0.00,0.02,-0.00,0.00,0.00,0.00,0.00,0.00


In [191]:
#  Which kernel function different the most?

# stat["sort"] = stat[""]
stat_kernel_fn = stat.sum(axis=1).to_frame()
# stat.sum(axis=1).sort_values(ascending=False)

In [193]:
stat_kernel_fn["sort"] = abs(stat_kernel_fn[0])

In [196]:
stat_kernel_fn.sort_values(by="sort",ascending=False)[0].to_frame()

,0
Demangled Name,
"void mul_mat_vec_q<(int)32, (int)4, block_q4_0, (int)2, &vec_dot_q4_0_q8_1>(const void *, const void *, float *, int, int)","-72,356.00"
"void mul_mat_vec_q<(int)32, (int)8, block_q8_0, (int)2, &vec_dot_q8_0_q8_1>(const void *, const void *, float *, int, int)","20,722.45"
"void k_argsort_f32_i32<(ggml_sort_order)1>(const float *, int *, int)","2,803.72"
"void soft_max_f32<(bool)1, (int)0, (int)0>(const float *, const float *, float *, int, int, float)","2,466.43"
"k_compute_batched_ptrs(const __half *, const __half *, char *, const void **, void **, long, long, long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, long, long)","1,877.36"
"void cpy_f32_f16<&cpy_1_f32_f16>(const char *, char *, int, int, int, int, int, int, int, int, int, int, int)","1,510.35"
turing_h1688gemm_256x64_ldg8_stages_32x1_tn,"1,502.01"
"void convert_unary<float, __half>(const void *, T2 *, int)","1,395.69"
"k_sum_rows_f32(const float *, float *, int)","1,378.98"


In [190]:
stat.sum(axis=1).to_frame()[0]

Demangled Name
k_compute_batched_ptrs(const __half *, const __half *, char *, const void **, void **, long, long, long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, unsigned long, long, long)            1,877.36
k_sum_rows_f32(const float *, float *, int)                                                                                                                                                                               1,378.98
quantize_q8_1(const float *, void *, int, int)                                                                                                                                                                              353.07
silu_f32(const float *, float *, int)                                                                                                                                                                                       919.41
turing_h1688gemm_256x64_ldg8_stages_32x1_tn                                  

In [155]:
#  Which metric is most representative?

stat.sum().sort_values(ascending=False)[:30]

# (Level 1 (L1)/Texture Cache, l1tex) LSU (Load Store Unit)

#  https://docs.nvidia.com/nsight-compute/NsightComputeCli/index.html
# shared_load_throughput	l1tex__data_pipe_lsu_wavefronts_mem_shared_op_ld.sum.per_second
# shared_load_transactions	l1tex__data_pipe_lsu_wavefronts_mem_shared_op_ld.sum
# lg	Local/Global memory

# fbpa	The FrameBuffer Partition is a memory controller which sits between the level 2 cache (LTC) and the DRAM. 
# dram	Device (main) memory, where the GPUs global and local memory resides.


# Full Metrics Units:
# https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#metrics-guide


l1tex__data_pipe_lsu_wavefronts_cmd_write.max                       93.35
l1tex__data_pipe_lsu_wavefronts_mem_lg_cmd_write.max                93.34
fbpa__dram_write_bytes.avg.pct_of_peak_sustained_active [%]         12.07
fbpa__dram_write_sectors.sum.pct_of_peak_sustained_active [%]       12.07
fbpa__dram_write_throughput.avg.pct_of_peak_sustained_active [%]    12.07
fbpa__dram_write_throughput.sum.pct_of_peak_sustained_active [%]    12.07
fbpa__dram_write_bytes.sum.pct_of_peak_sustained_active [%]         12.07
fbpa__dram_write_sectors.avg.pct_of_peak_sustained_active [%]       12.07
fbpa__dram_write_bytes.sum.per_cycle_active [byte/cycle]            12.06
dram__sectors_write.sum.per_cycle_active [sector/cycle]             12.02
fbpa__dram_write_bytes.sum.pct_of_peak_sustained_frame [%]          11.99
fbpa__dram_write_sectors.avg.pct_of_peak_sustained_region [%]       11.99
fbpa__dram_write_sectors.avg.pct_of_peak_sustained_frame [%]        11.99
fbpa__dram_write_sectors.avg.pct_of_pe

In [152]:
stat.to_csv("./stat.csv")

In [156]:
stat.sum().sort_values(ascending=False).to_csv("./stat_metrics.csv")

In [180]:
kw = [
    # "Throughput",
    "Tensor",
    "inst_executed",
    "integer_",
    "int8",
    "int4",
    "fp16",
    "fp32",
    "fp64",
    
]

skip = [
    "fbpa",
    "dram",
    
]
def in_kw(col):
    for s in skip:
        if s.lower() in col.lower():
            return False
    for k in kw:
        if k.lower() in col.lower():
            return True
            
    return False

In [177]:
metrics_stat = stat.sum().to_frame()


In [178]:
metrics_stat # sum of all the kernel function calls 

,0
device__attribute_architecture,1.00
device__attribute_async_engine_count,1.00
device__attribute_can_map_host_memory,1.00
device__attribute_can_tex2d_gather,1.00
device__attribute_can_use_64_bit_stream_mem_ops,1.00
...,...
tpc__warps_active_shader_cs_realtime.sum.per_cycle_active [warp],1.44
tpc__warps_active_shader_cs_realtime.sum.per_cycle_elapsed [warp],1.44
tpc__warps_active_shader_cs_realtime.sum.per_cycle_in_frame [warp],1.44
tpc__warps_active_shader_cs_realtime.sum.per_cycle_in_region [warp],1.44


In [201]:
metrics_stat.loc[[c for c in metrics_stat.index if in_kw(c)]].sort_values(by=0, ascending=False)[:30]


# **smsp** Each SM is partitioned into four processing blocks, called SM sub partitions. The
# SM sub partitions are the primary processing elements on the SM. A sub partition
# manages a fixed size pool of warps.




,0
smsp__pipe_tensor_cycles_active.max.pct_of_peak_sustained_active [%],5.17
smsp__pipe_tensor_op_hmma_cycles_active.max.pct_of_peak_sustained_active [%],5.17
smsp__thread_inst_executed_pred_off.min.pct_of_peak_sustained_active [%],4.69
smsp__thread_inst_executed_pred_off.min.per_cycle_active [inst/cycle],4.36
smsp__pipe_tensor_op_hmma_cycles_active.max.pct_of_peak_sustained_elapsed [%],4.33
smsp__pipe_tensor_op_hmma_cycles_active.max.pct_of_peak_sustained_region [%],4.33
smsp__pipe_tensor_cycles_active.max.pct_of_peak_sustained_elapsed [%],4.33
smsp__pipe_tensor_cycles_active.max.pct_of_peak_sustained_frame [%],4.33
smsp__pipe_tensor_op_hmma_cycles_active.max.pct_of_peak_sustained_frame [%],4.33
smsp__pipe_tensor_cycles_active.max.pct_of_peak_sustained_region [%],4.33


In [200]:
# stat2 = selected_nonzero_diff.div( abs(selected_q4_agg.add(selected_q8_agg, fill_value=0.0)), fill_value=0.0)
# stat2

In [202]:
# New Stat, consider inflow and outflow (positives and negatives)
metrics_stat = stat.sum().to_frame()


In [203]:
metrics_stat.loc[[c for c in metrics_stat.index if in_kw(c)]].sort_values(by=0, ascending=False)[:30]


,0
smsp__pipe_tensor_op_hmma_cycles_active.max.pct_of_peak_sustained_active [%],3.17
smsp__pipe_tensor_cycles_active.max.pct_of_peak_sustained_active [%],3.17
smsp__pipe_tensor_cycles_active.max.pct_of_peak_sustained_elapsed [%],2.33
smsp__pipe_tensor_op_hmma_cycles_active.max.pct_of_peak_sustained_region [%],2.33
smsp__pipe_tensor_cycles_active.max.pct_of_peak_sustained_region [%],2.33
smsp__pipe_tensor_cycles_active.max.pct_of_peak_sustained_frame [%],2.33
smsp__pipe_tensor_op_hmma_cycles_active.max.pct_of_peak_sustained_frame [%],2.33
smsp__pipe_tensor_op_hmma_cycles_active.max.pct_of_peak_sustained_elapsed [%],2.33
smsp__pipe_tensor_op_hmma_cycles_active.max.per_second [cycle/usecond],2.07
smsp__pipe_tensor_cycles_active.max.per_second [cycle/usecond],2.07
